#Loading Data

In [0]:
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

In [7]:
!pip install category_encoders

    100% |████████████████████████████████| 61kB 2.4MB/s 


In [2]:
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train_features = pd.read_csv('/content/gdrive/My Drive/train_features.csv')
train_labels = pd.read_csv('/content/gdrive/My Drive/train_labels.csv')
test_features = pd.read_csv('/content/gdrive/My Drive/test_features.csv')
submission = pd.read_csv('/content/gdrive/My Drive/sample_submission.csv')

In [33]:
train_features.shape, train_labels.shape, test_features.shape

((59400, 40), (59400, 2), (14358, 40))

In [0]:
test_features.head()

In [0]:
test_features.isnull().sum()

In [0]:
train_features.funder.value_counts()

In [0]:
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [0]:
train_features.head().T

In [0]:
y_train = train_labels['status_group']
X_train = train_features.drop(columns='id')
X_test  = test_features.drop(columns='id')

In [0]:
train_labels.head()

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((44550, 39), (14850, 39), (44550,), (14850,))

In [0]:
y_train.value_counts(normalize=True)

#Modeling

In [0]:
X_train_numeric = wrangle_OneCode(X_train)
X_val_numeric = wrangle_OneCode(X_val)
X_test_numeric = wrangle_OneCode(X_test)

In [15]:
X_train_numeric.shape, X_val_numeric.shape,X_test_numeric.shape

((44550, 29), (14850, 29), (14358, 29))

In [0]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    StandardScaler(),
    #LogisticRegression(solver='lbfgs',multi_class='multinomial',max_iter=10000)
    KNeighborsClassifier()
)


In [0]:
X_train_numeric.head()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
12207,0.0,0,34.032267,-8.852410,0,12,7,0,0
49560,0.0,1388,36.698512,-2.722407,0,2,6,781,1978
10189,0.0,1285,37.011981,-3.207829,0,3,7,1,2001
39485,0.0,0,30.846356,-2.958824,0,18,30,0,0
22354,0.0,0,33.219071,-2.902018,0,19,7,0,0


In [0]:
X_train.head().T

In [0]:
X_train.water_quality.value_counts()

soft                  38059
salty                  3694
unknown                1395
milky                   593
coloured                383
salty abandoned         260
fluoride                154
fluoride abandoned       12
Name: water_quality, dtype: int64

In [0]:
X_train_q=pd.get_dummies(X_train,columns=['quality_group'])

In [0]:
y_train.head()

In [0]:
X_train_q['y']=y_train

In [0]:
X_train_q.columns

Index(['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quantity', 'quantity_group', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'quality_group_colored',
       'quality_group_fluoride', 'quality_group_good', 'quality_group_milky',
       'quality_group_salty', 'quality_group_unknown', 'y'],
      dtype='object')

In [0]:
X_try=X_train_q[['quality_group_colored',
       'quality_group_fluoride', 'quality_group_good', 'quality_group_milky',
       'quality_group_salty', 'quality_group_unknown', 'y']]

In [0]:
X_try.y.value_counts()
X_try.y=X_try.y.map(status_map)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [0]:
X_try.corr().y.sort_values()

In [0]:
X_train.select_dtypes(exclude=np.number).nunique()

In [0]:
X_train[low_cardinality].isnull().sum()

In [0]:
X_train=pd.get_dummies(X_train,columns=low_cardinality)

In [0]:
X_train.shape

(44550, 177)

In [0]:
status_map={'non functional':0,'functional needs repair':1,'functional':2}

In [0]:
y_train = y_train.replace(status_map)

In [0]:
X_try.corr().y.sort_values()

In [0]:
X_train.isnull().sum()

In [0]:
X_train = X_train.drop('scheme_name',axis =1)

In [0]:
X_train = X_train.dropna()

In [0]:
X_train_numeric.quantity

In [51]:
X_train_numeric.isnull().sum()

amount_tsh                   0
installer                 2757
num_private                  0
basin                        0
region                       0
region_code                  0
district_code                0
recorded_by                  0
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group            19234
quantity                     0
quantity_group               0
source                       0
source_type                  0
source_class                 0
waterpoint_type              0
waterpoint_type_group        0
quality_group_square     19234
dtype: int64

In [50]:
m = pipeline.fit(X_train_numeric,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


ValueError: ignored

In [0]:
X_train.quality_group.value_counts()

good        38129
salty        3907
unknown      1388
milky         592
colored       362
fluoride      172
Name: quality_group, dtype: int64

In [0]:
def wrangle(X):
  X = X.copy()
  low_cardinality = ['basin','region',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quantity', 'source', 'source_type','quantity_group',
       'source_class', 'waterpoint_type', 'waterpoint_type_group']
  
  X=pd.get_dummies(X,columns=low_cardinality)
  quality_group_map = {'unknown':0,'salty':1,'milky':2,'colored':3,'fluoride':4,'good':5}
  X['quality_group'] =  X['quality_group'].map(quality_group_map)
  X['quality_group_square']=X['quality_group']**2
  
  X = X.select_dtypes(np.number)
  
  
  return X

def wrangle_OneCode(X):
  X = X.copy()
  low_cardinality = ['basin','region',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quantity', 'source', 'source_type','quantity_group',
       'source_class', 'waterpoint_type', 'waterpoint_type_group']
  
  install_less_than_300 =  X['installer'].value_counts()[X['installer'].value_counts()<300].index
  install_less_than_300= list(install_less_than_300)
  X[X['installer'].isin(install_less_than_300)]='Other'
  
  X.installer.value_counts()
  
  #X=pd.get_dummies(X,columns=low_cardinality)
  quality_group_map = {'unknown':0,'salty':1,'milky':2,'colored':3,'fluoride':4,'good':5}
  X['quality_group'] =  X['quality_group'].map(quality_group_map)
  X['quality_group_square']=X['quality_group']**2
  
  high_cardinality =list(X.select_dtypes(exclude=np.number).nunique()[X.select_dtypes(exclude=np.number).nunique()>100].index)
  X=X.drop(high_cardinality, axis=1)
  X=X.drop(['public_meeting','scheme_management','permit'], axis=1)
  
  #X['construction_year'].replace({0:X['construction_year'].mean()})
  #print(X.isnull().sum())
  #print(X.columns)
  
  #enc = OneHotEncoder(handle_unknown='ignore')
  #X=enc.fit_transform(X)
  #print(type(X))
  
  
  #X = X.select_dtypes(np.number)
  
  
  return X

In [0]:
list(X_train.select_dtypes(exclude=np.number).nunique()[X_train.select_dtypes(exclude=np.number).nunique()>100].index)

In [0]:
X_train['construction_year'].replace({0:X_train['construction_year'].mean()})

In [13]:
wrangle_OneCode(X_train).isnull().sum()

amount_tsh               0
gps_height               0
longitude                0
latitude                 0
num_private              0
basin                    0
region                   0
region_code              0
district_code            0
population               0
recorded_by              0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
quality_group_square     0
dtype: int64

In [20]:
y_pred=m.predict(X_val_numeric)

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [0]:
y_val=y_val.map(status_map)

In [22]:
y_pred

array([2, 0, 2, ..., 2, 0, 0])

In [0]:
y_val

In [24]:
accuracy_score(y_val,y_pred)

0.7653198653198653

In [0]:
pd.Series(y_pred).value_counts()

2    8233
0    5522
1     603
dtype: int64

#Testing

In [0]:
y_pred=m.predict(X_test_numeric)

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [0]:
y_pred = pd.Series(y_pred)

In [0]:
status_map_r = {0:'non functional',1:'functional needs repair',2:'functional'}

In [0]:
y_pred = y_pred.replace(status_map_r)

In [0]:
y_pred.value_counts()

functional                 8117
non functional             5503
functional needs repair     738
dtype: int64

In [0]:
submission['status_group']=y_pred

In [0]:
submission.to_csv('test')

In [0]:
from google.colab import files

submission.to_csv('test_labels.csv',index=False)
files.download('test_labels.csv')